<a href="https://colab.research.google.com/github/acollin19/comp551/blob/main/A2/assignment2_group_24.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment 2

## Process:

### Task 1: Data Processing  


1. IMDB Reviews
2. 20 news groups: a multi-class labelled textual dataset


### Task 2: Implement Logistic and Multiclass classifiers

1. Logistic Regression
2. Multiclass Regression
3. Comparisons using ROC curve


### Task 3: Running Experiments



1.    Most positive/negative Z-scores
2.   Implement:  
    * Binary classification on the IMDB Reviews

  *   Multi-class classification on the 20 news group dataset

3. Binary classification AUROC on IMDB data
  *   Logistic Regression 
  *   KNN

4. Multiclass classification accuracy

5. Compare Accuracy of Models (Plot)







In [5]:
import pandas as pd
import numpy as np
from sklearn.datasets import load_svmlight_file

from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

# Multiclass Regression and ROC curve comparisons
from sklearn import model_selection
from matplotlib import pyplot as plt
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.linear_model import LogisticRegression as sk_LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier

In [6]:
!curl -O https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
!tar -xf aclImdb_v1.tar.gz
!ls aclImdb
!ls aclImdb/test
!ls aclImdb/train

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 80.2M  100 80.2M    0     0  42.7M      0  0:00:01  0:00:01 --:--:-- 42.7M
imdbEr.txt  imdb.vocab	README	test  train
labeledBow.feat  neg  pos  urls_neg.txt  urls_pos.txt
labeledBow.feat  pos	unsupBow.feat  urls_pos.txt
neg		 unsup	urls_neg.txt   urls_unsup.txt


In [ ]:
# IMDB dataset preprocession
'''
# Needs cleaning 
- removal of rare and stopwords
- choose the top D ∈ [100, 1000] features by their absolute z-score  using simple regression
''' 

In [7]:
# Loading 20 news group dataset
categories=['rec.sport.hockey','sci.electronics','talk.politics.guns','sci.space']
twenty_train = fetch_20newsgroups(subset='train', categories=categories, remove=(['headers', 'footers', 'quotes']), shuffle=True, random_state=42)

'''
Also needs cleaning
- filter out rare words, stopwords, and words that are not relevant to any of the 4 class labels.
- MI to select the top M ∈ [10, 100] feature words per class and take the union of all top feature words to train your multiclass model.

'''

# Task 1 : Data processing



## Task 1.2 : IMDB

 - X contain features based on the words in the movie reviews
 - y contain labels for whether the review sentiment is positive 1 or negative -1

In [8]:
# IMDB train df
 
IMDB_X_train, IMDB_y_train = load_svmlight_file('aclImdb/train/labeledBow.feat')
IMDB_X_test, IMDB_y_test = load_svmlight_file('aclImdb/test/labeledBow.feat') 

IMDB_y_train[IMDB_y_train < 5] = -1.0
IMDB_y_train[IMDB_y_train >= 5] = 1.0


# QUESTION TO ANSWER: Do they make sense for calling a movie good and bad, respectively?


## Task 1.2 : Twenty Train

Text preprocessing, tokenizing and filtering of stopwords are all included in the CountVectorizer function

In [9]:
# Turn the text content into feature vectors
count_vect = CountVectorizer() 
X_train_counts = count_vect.fit_transform(twenty_train.data) # builds a dict of features and transforms docs to feature vectors
X_train_counts.shape

# Occurences to frequences : TF_IDF

tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

In [10]:
X = twenty_train.data # Test content

# Turn the text content into feature vectors
count_vect = CountVectorizer() 
X_train_counts = count_vect.fit_transform(twenty_train.data) # builds a dict of features and transforms docs to feature vectors
X_train_counts.shape

# Occurences to frequences : TF_IDF

tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
X = X_train_tfidf

y_cat = twenty_train.target
# convert array of class indices to one-hot encoded array
y = np.zeros((y_cat.size, y_cat.max() + 1))
y[np.arange(y_cat.size), y_cat] = 1

print(X)

  (0, 30948)	0.14040989982008775
  (0, 30906)	0.08405138460178864
  (0, 30883)	0.09530400819467387
  (0, 30295)	0.10290468881809638
  (0, 30251)	0.05264064004143937
  (0, 28527)	0.10189915843591614
  (0, 28304)	0.09697317574876095
  (0, 28141)	0.08274417530859261
  (0, 28131)	0.025242498432943575
  (0, 27975)	0.07298610652089194
  (0, 27906)	0.05419820732883213
  (0, 27860)	0.04363340430813677
  (0, 27841)	0.05366514697429806
  (0, 27834)	0.15692196174342432
  (0, 27829)	0.08962777594902022
  (0, 27822)	0.09928182630437254
  (0, 27555)	0.12458624583194353
  (0, 26949)	0.09928182630437254
  (0, 26386)	0.14441450882235457
  (0, 26327)	0.1334455497312062
  (0, 26326)	0.15800080004504655
  (0, 26046)	0.04679870001486382
  (0, 26027)	0.15800080004504655
  (0, 25583)	0.059145254902454356
  (0, 24869)	0.14004069997244845
  :	:
  (2329, 30433)	0.06815319170856916
  (2329, 27883)	0.21301289255201286
  (2329, 26088)	0.16672798990930832
  (2329, 22608)	0.2353383104271898
  (2329, 20613)	0.0476772

# Task 2 : Implementing Classes

## Task 2.1 : Logistic Regression Class

In [11]:
# Logistic function
z = np.linspace(-5,5,100)
logistic = lambda z: 1./ (1 + np.exp(-z))

# From class Logistic Regression collab
class LogisticRegression:
    
    def __init__(self, add_bias=True, learning_rate=.1, epsilon=1e-4, max_iters=1e5, verbose=False):
        self.add_bias = add_bias
        self.learning_rate = learning_rate
        self.epsilon = epsilon                        #to get the tolerance for the norm of gradients 
        self.max_iters = max_iters                    #maximum number of iteration of gradient descent
        self.verbose = verbose

    def gradient(self, x, y):
        N,D = x.shape
        yh = logistic(np.dot(x, self.w))    # predictions  size N
        grad = np.dot(x.T, yh - y)/N        # divide by N because cost is mean over N points
        return grad
        
    def fit(self, x, y):
        if x.ndim == 1:
            x = x[:, None]
        if self.add_bias:
            N = x.shape[0]
            x = np.column_stack([x,np.ones(N)])
        N,D = x.shape
        self.w = np.zeros(D)
        g = np.inf 
        t = 0
        # the code snippet below is for gradient descent
        while np.linalg.norm(g) > self.epsilon and t < self.max_iters:
            g = self.gradient(x, y)
            self.w = self.w - self.learning_rate * g 
            t += 1
        
        if self.verbose:
            print(f'terminated after {t} iterations, with norm of the gradient equal to {np.linalg.norm(g)}')
            print(f'the weight found: {self.w}')
        return self
    
    def predict(self, x):
        if x.ndim == 1:
            x = x[:, None]
        Nt = x.shape[0]
        if self.add_bias:
            x = np.column_stack([x,np.ones(Nt)])
        yh = logistic(np.dot(x,self.w))            #predict output
        return yh

# LogisticRegression.gradient = gradient  

In [ ]:
# Applying Logistic Regression on the IMDB data

model = LogisticRegression(verbose=True, add_bias=False)
fit = model.fit(IMDB_X_train, IMDB_y_train) # DOESNT WORK 
prediction = fit.predict(IMDB_X_train)

lin_coefficients= pd.DataFrame(fit.w[:(len(fit.w)-1)]).transpose()
print(lin_coefficients.to_string(index=False))

# plt.plot(X_train, y_train, '.', label='dataset')
# plt.plot(X_train, prediction, 'g', alpha=.5, label='predictions')
# plt.xlabel('x')
# plt.ylabel(r'$y$')
# plt.legend()
# # plt.show()
# plt.savefig('IMDB_prediction.png', bbox_inches="tight", dpi=300)

In [ ]:
'''
Check Gradient using small perturbation, 
monitor the cross-entropy as a function of iteration, 
and report your finding on both datasets
'''



In [15]:
# Extra - Applying KNN on IMDB
knn = KNeighborsClassifier()
knn.fit(IMDB_X_train[:,:89523], IMDB_y_train)

pred = knn.predict(IMDB_X_test)[0]

print("Test Prediction for 0 : ", pred)

Test Prediction for 0 :  -1.0


## Task 2.2 : Multiclass Regression Class

In [9]:
# From class Multiclass Regression collab
class Multinomial_logistic:
    def __init__(self, nFeatures, nClasses):
        self.W = np.random.rand(nFeatures, nClasses)

    def predict(self, X):
        y_pred = np.exp(np.matmul(X, self.W))
        return y_pred / y_pred.sum(axis=1).reshape(X.shape[0], 1)

    def grad(self, X, y):
        return np.matmul(X.transpose(), self.predict(X) - y)

    def ce(self, X, y):
        return -np.sum(y * np.log(self.predict(X)))

    # modify it to add stopping criteria (what can you think of?)
    def fit(self, X, y, X_valid=None, y_valid=None, lr=0.005, niter=100):
        losses_train = np.zeros(niter)
        losses_valid = np.zeros(niter)
        for i in range(niter):
            self.W = self.W - lr * self.grad(X, y)
            loss_train = self.ce(X, y)
            losses_train[i] = loss_train
            if X_valid is not None and y_valid is not None:
                loss_valid = self.ce(X_valid, y_valid)
                losses_valid[i] = loss_valid
                print(f"iter {i}: {loss_train:.3f}; {loss_valid:.3f}")
            else:
                print(f"iter {i}: {loss_train:.3f}")
        return losses_train, losses_valid

    def check_grad(self, X, y):
        N, C = y.shape
        D = X.shape[1]

        diff = np.zeros((D, C))

        W = self.W.copy()

        for i in range(D):
            for j in range(C):
                epsilon = np.zeros((D, C))
                epsilon[i, j] = np.random.rand() * 1e-4

                self.W = self.W + epsilon
                J1 = self.ce(X, y)
                self.W = W

                self.W = self.W - epsilon
                J2 = self.ce(X, y)
                self.W = W

                numeric_grad = (J1 - J2) / (2 * epsilon[i, j])
                derived_grad = self.grad(X, y)[i, j]

                diff[i, j] = np.square(derived_grad - numeric_grad).sum() / \
                             np.square(derived_grad + numeric_grad).sum()

        # print(diff)
        return diff.sum()

    def evaluate(self, y, y_pred):
      accuracy = sum(y_pred.argmax(axis=1) == y.argmax(axis=1))
      accuracy = accuracy / y.shape[0]

      return accuracy

In [ ]:
# Applying Multiclass Regression on the 4-class prediction from the 20-news-group data

# Split the data into training, validation, and testing
X_train, X_test, y_train, y_test = model_selection.train_test_split(
    X, y, test_size = 0.33, random_state=1, shuffle=True)

X_train, X_valid, y_train, y_valid = model_selection.train_test_split(
    X_train, y_train, test_size = 0.5, random_state=1, shuffle=True)


## Task 2.3 : Method comparison using ROC curve

In [ ]:
models = [KNeighborsClassifier(),
          DecisionTreeClassifier(),
          sk_LogisticRegression()]

perf = {}

# our implementation is slow you may try sklearn version
logitreg = LogisticRegression(max_iters=1e3)
fit = logitreg.fit(X_train, y_train)
y_test_prob = fit.predict(X_test)
fpr, tpr, _ = roc_curve(y_test, y_test_prob)
auroc = roc_auc_score(y_test, y_test_prob)
perf["LogisticRegression (ours)"] = {'fpr':fpr, 'tpr':tpr, 'auroc':auroc}

for model in models:
    fit = model.fit(X_train, y_train)
    y_test_prob = fit.predict_proba(X_test)[:,1]
    fpr, tpr, _ = roc_curve(y_test, y_test_prob)
    auroc = roc_auc_score(y_test, y_test_prob)
    if type(model).__name__ == "LogisticRegression":
        perf["LogisticRegression (sklearn)"] = {'fpr':fpr,'tpr':tpr,'auroc':auroc}
    else:
        perf[type(model).__name__] = {'fpr':fpr,'tpr':tpr,'auroc':auroc}


plt.clf()
i = 0
for model_name, model_perf in perf.items():
    plt.plot(model_perf['fpr'], model_perf['tpr'],label=model_name)
    plt.text(0.4, i+0.1, model_name + ': AUC = '+ str(round(model_perf['auroc'],2)))
    i += 0.1

plt.plot([0, 1], [0, 1], color="navy", lw=2, linestyle="--")
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])

plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title('ROC in predicting Titanic survivor')
plt.legend(bbox_to_anchor=(1.04, 0.5), loc="upper left")
# plt.show()
plt.savefig("roc_curve.png", bbox_inches='tight', dpi=300)
# plt.close()

# Task 3 : Running Experiments

## Task 3.1 : Z-Scores
Report top 10 features with most positive/negative z-scores on the IMDB data using simple linear regression on the movie rating scores.

## Task 3.2.1: ROC and AUROC for Binary Classification

## Task 3.2.2 : Classification Accuracy for Multi-class Prediction